## Rainfall Data Extraction from GPM

Global Precipitation Measurement (GPM) is an international satellite mission to provide next-generation observations of rain and snow worldwide every three hours. For our project, we needed to extract amount of historic rainfall in a given region during a specific time-window in a day. 

This notebook shows you one can use the Google Earth Engine API to extract this information from the satellite data.



In [ ]:
import ee

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

We need to know what amount of rainfall was recorded at each of the 198 wards of the city. `bangalore` contains the ward boundaries shapefile uploaded to Google Earth Engine

In [ ]:
gpm = ee.ImageCollection("NASA/GPM_L3/IMERG_V06"),
bangalore = ee.FeatureCollection("users/vishnuprasadjs21/bangalore_ward");

We need to extract rainfall data for the following time-periods

* midnight-7am
* 7am - 10am
* 10am - 4pm
* 4pm - 7pm
* 7pm - mignight

We need to do this for all days of the Quarter 1, 2020. 

In [ ]:
periodlist = ee.List([[0, 7], [7,10], [10, 16],[16,19],[19,24]])
year = 2020;
day_of_year = ee.List.sequence(1,91);
start_date = ee.Date.fromYMD(2020, 1, 1);

In [ ]:
gpm

In [ ]:
def period_rain(period,day):

    s = ee.List(period).get(0);
    e = ee.List(period).get(1);
    start = start_date.advance(ee.Number(day).subtract(1),'day').advance(s,'hour')
    end = start_date.advance(ee.Number(day).subtract(1),'day').advance(e,'hour')
    filtered = gpm.filter(ee.Filter.calendarRange(day,day,'DAY_OF_YEAR')).filter(ee.Filter.date(start,end)).reduce(ee.Reducer.sum()).select('precipitationCal_sum').divide(2)
    
    stats =filtered.reduceRegions(collection= bangalore,reducer= ee.Reducer.mean(),scale= 5000)
          
    dup = stats.map(lambda i : i.set({'year':year,'day_of_year': day,'Time_period':period,'Precipitation_sum':i.get('mean')}).setGeometry())
    
    return dup

In [ ]:
rainfall = day_of_year.map(lambda day : periodlist.map(lambda period: period_rain(period,day) ).flatten())

The output is a nested collection of data for each period, for each day of the quarter. Use `flatten()` to create a flattened collection that can be exported.

In [ ]:
output = ee.FeatureCollection(rainfall.flatten()).flatten()

In [ ]:
output.first().getInfo()

{'geometry': None,
 'id': '0_0000000000000000008a',
 'properties': {'DISPLAY_NA': '0 3rd Cross Road, 4N Block, Ramchandrapuram, Srirampura, Bengaluru',
  'MOVEMENT_I': '102',
  'Precipitation_sum': 0,
  'Time_period': [0, 7],
  'WARD_NAME': 'Okalipuram',
  'WARD_NO': '96',
  'day_of_year': 1,
  'mean': 0,
  'year': 2020},
 'type': 'Feature'}

In [ ]:
task = ee.batch.Export.table.toDrive( collection= output,folder='earthengine',description='rainfall_bg',fileFormat= 'csv')

In [ ]:
task.start()

In [ ]:
task.status()

{'creation_timestamp_ms': 1608870931013,
 'description': 'rainfall_bg',
 'id': 'DPCOETKW7X26NEF37WP7LIPF',
 'name': 'projects/earthengine-legacy/operations/DPCOETKW7X26NEF37WP7LIPF',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_FEATURES',
 'update_timestamp_ms': 1608870931013}